In [1]:
import numpy as np

import scipy as sp
from scipy import spatial

import matplotlib.pyplot as plt # side-stepping mpl backend

from nltk.stem.lancaster import LancasterStemmer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import pairwise
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn import cross_validation
import heapq
import string
import re

from nltk.corpus import stopwords
from collections import defaultdict


vectorizer = CountVectorizer()
st = LancasterStemmer()

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open("TestData/train.dat", "r") as fh:
    #with open("TestData/Test/training_out.dat", "r") as fh:
    linesOfTrainData = fh.readlines()
print(len(linesOfTrainData))

#with open("TestData/format.dat", "r") as fh:
with open("TestData/Test/format_out.dat", "r") as fh:
    linesOfFormat = fh.readlines()
print(len(linesOfFormat))

with open("TestData/test.dat", "r") as fh:
#with open("TestData/Test/test_out.dat", "r") as fh:
    linesOfTestData = fh.readlines()
print(len(linesOfTestData))


18506
0
18506


In [4]:
##stopwords and stemming for Training

print ("First line before cleanup from Training Data: ",linesOfTrainData[0])

stops = set(stopwords.words('english'))

linesOfTrainDataAfterPreProcessing=[]
for line in linesOfTrainData:
    newLine = []
    for w in line.split():
        if w.lower()not in stops:
            newLine.append(w)     #for stopwords  
    linesOfTrainDataAfterPreProcessing.append(" ".join(newLine))


linesOfTrainDataAfterSteming = []
for line in linesOfTrainDataAfterPreProcessing:
    newLine = line
    for w in newLine.split():
            newLine = newLine.replace(w, st.stem(w)) # for stemming
    linesOfTrainDataAfterSteming.append(newLine)
    
#Not working    
linesOfTrainDataAfterStemingWithoutPunctuation = []
for line in linesOfTrainDataAfterSteming:
    newLine = line
    for w in newLine.split():
         if w in string.punctuation:
            newLine = newLine.translate(string.punctuation)
            #newLine = newLine.replace(w,re.sub(r'[^\w\s]()','',w)) # for Punctuation 
    linesOfTrainDataAfterStemingWithoutPunctuation.append(newLine)    


linesOfTrainData = linesOfTrainDataAfterStemingWithoutPunctuation

print ("\n\nFirst line after cleanup from Training Data: ",linesOfTrainData[0])

First line before cleanup from Training Data:  +1	This book is such a life saver.  It has been so helpful to be able to go back to track trends, answer pediatrician questions, or communicate with each other when you are up at different times of the night with a newborn.  I think it is one of those things that everyone should be required to have before they leave the hospital.  We went through all the pages of the newborn version, then moved to the infant version, and will finish up the second infant book (third total) right as our baby turns 1.  See other things that are must haves for baby at [...]



First line after cleanup from Training Data:  +1 book lif saver. help abl go back track trends, answ pedy questions, commun diff tim night newborn. think on thing everyon requir leav hospital. went pag newborn version, mov inf version, fin second inf book (third total) right baby turn 1. see thing must hav baby [...]


In [6]:
##stopwords and stemming for Testdata

print ("First line before cleanup from Test Data: ",linesOfTestData[0])

stops = set(stopwords.words('english'))

linesOfTestDataAfterPreProcessing=[]
for line in linesOfTestData:
    newLine = []
    for w in line.split():
        if w.lower()not in stops:
            newLine.append(w)       
    linesOfTestDataAfterPreProcessing.append(" ".join(newLine)) # for identifting words
    

linesOfTestDataAfterSteming = []
for line in linesOfTestDataAfterPreProcessing:
    newLine = line
    for w in newLine.split():
            newLine = newLine.replace(w, st.stem(w)) # for stemming
    linesOfTestDataAfterSteming.append(newLine)
    
linesOfTestDataAfterStemingWithoutPunctuation = []
for line in linesOfTestDataAfterSteming:
    newLine = line
    for w in newLine.split():
         if w in string.punctuation:
            newLine = newLine.translate(string.punctuation)
            #newLine = newLine.replace(w,re.sub(r'[^\w\s]()','',w)) # for Punctuation 
    linesOfTestDataAfterStemingWithoutPunctuation.append(newLine)
    
    

linesOfTestData = linesOfTestDataAfterStemingWithoutPunctuation

print ("\n\nFirst line after cleanup from Test Data: ",linesOfTestData[0])

First line before cleanup from Test Data:  Perfect for new parents. We were able to keep track of baby's feeding, sleep and diaper change schedule for the first two and a half months of her life. Made life easier when the doctor would ask questions about habits because we had it all right there!



First line after cleanup from Test Data:  perfect new parents. abl keep track baby's feeding, sleep diap chang schedule first two half month lif. mad lif easy doct would ask quest habit right there!


In [33]:
#     vectorizer = CountVectorizer(analyzer = "word",   \
#                              tokenizer = None,    \
#                              preprocessor = None, \
#                              stop_words = None,   \
#                              max_features = 5000)
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 50, stop_words = 'english')
selector.fit(linesOfTrainData, linesOfTestData)     
linesOfTrainData_Transformed = vectorizer.fit_transform(linesOfTrainData)
#print(linesOfTrainData_Transformed[:1])

NameError: name 'selector' is not defined

In [29]:

linesOfTestData_Transformed = vectorizer.transform(linesOfTestData)
#print(test_data_features[:1])


In [30]:
feature_names = vectorizer.get_feature_names() 
len(feature_names)


2173

In [31]:
def CalculateCosine(vt,vs):
        cosineSimilarityValue = cosine_similarity(vt,vs)
        return cosineSimilarityValue

In [32]:
cosineSimilarityValue = CalculateCosine(linesOfTestData_Transformed,linesOfTrainData_Transformed)


ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 2173 while Y.shape[1] == 2181

In [51]:

f = open('TestData/Test/format_out.dat', 'w')
count = 0
for row in cosineSimilarityValue:

    
    #kneighbours = heapq.nlargest(5, row)
    k=72
    partitioned_row_byindex = np.argpartition(-row, k)  
    similar_index = partitioned_row_byindex[:k]
    
    #print(similar_index)
    
    neighbourReviewTypeList = []
    neighbourReviewTypeNegative = 0
    neighbourReviewTypePositive = 0
    
    #print("@@@@@",+count)
    for index in similar_index:

        if linesOfTrainData[index].strip()[0] == '-':
            #neighbourReviewTypeList.append("-1")
            neighbourReviewTypeNegative+=1
        elif linesOfTrainData[index].strip()[0] == '+':
            #neighbourReviewTypeList.append("+1")
            neighbourReviewTypePositive+=1
            
    
    if neighbourReviewTypeNegative > neighbourReviewTypePositive:
        f.write('-1\n')
        count+=1
    else:
        f.write('+1\n')
        count+=1

print("count : ",count)
print("--The End--")

count :  18506
--The End--


In [5]:
# get a frequency count for all words in the Training docs



wordsInTrainingSet = set()
for d in linesOfTrainData:
    for w in d.split():
        if w == "+1" or w == "-1":
            continue
        else:
            if w not in wordsInTrainingSet:
                wordsInTrainingSet.add(w)
                #wordsList.append(w)
            #else:
             #   wordsInTrainingSet[w] += 1
print("Number of unique words: %d." % len(wordsInTrainingSet))


Number of unique words: 63829.
